# Optimization - fine tuning in Initial Space

[1. Optimize LSH](#1-optimize-lsh)

[2. Optimize Hypercube](#2-optimize-hypercube)

[3. Optimize GNNS](#3-optimize-gnns)

[4. Optimize MRNG](#4-optimize-mrng)

[5. Optimize NSG](#5-optimize-nsg)

[6. Results](#6-results)

[7. Conclusions](#7-conclusions)

# Import libraries

In [ ]:
import numpy as np

import pandas

import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

from params import lsh_test, hypercube_test, gnn_test, mrng_test, nsg_test

In [ ]:
input_path = b'MNIST/input.dat'
query_path = b'MNIST/query.dat'

n = 60000

# 3. Optimize GNNS

To skip logs, click [here](#visualize-gnns-study-results).

In [ ]:
def objective_gnns(trial):
    param_dict = {'k': trial.suggest_int('k', 40, 100)}
    param_dict.update({'E': trial.suggest_int('E', 40, param_dict['k'])})
    param_dict.update({'R': trial.suggest_int('R', 1, 10)})

    print("Trial params", param_dict)

    average_time, aaf = gnn_test(input_path, query_path, queries_num=100, **param_dict, N=5)

    return aaf.value, average_time.value

In [ ]:
%%time
gnns_study = optuna.create_study(study_name='gnns', directions=['minimize', 'minimize'])
gnns_study.optimize(objective_gnns, n_trials=50, n_jobs=-1)
print("-------------------- Best trials --------------------")
trials = sorted(gnns_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

In [ ]:
df = gnns_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['values_0', 'values_1'])
df_sorted = df_sorted.sort_values(by=['values_0', 'values_1'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

## Visualize GNNS study results

In [ ]:
plot_pareto_front(gnns_study, target_names=['aaf', 'average_time'])

In [ ]:
plot_optimization_history(gnns_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_optimization_history(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_slice(gnns_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_slice(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')